In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
from datetime import datetime

def scrap_saison_complete(saison: str, max_journee: int):
    """
    Scrap toutes les journées de Ligue 1, Ligue 2 et National pour une saison donnée
    et enregistre le tout dans un seul CSV trié chronologiquement.

    Args:
        saison (str): saison au format "YYYY-YYYY" (ex: "2025-2026")
    """

    ligues_map = {
        "ligue1": "ligue-1",
        "ligue2": "ligue-2",
        "national": "national"
    }

    dossier = "C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Résultats"

    os.makedirs(dossier, exist_ok=True)
    name_file = f"{dossier}/{saison}_all.csv"

    all_matches = []

    for ligue, ligue_url in ligues_map.items():
        journee = 1
        while journee <= max_journee:
            url = f"https://www.mondefootball.fr/calendrier/fra-{ligue_url}-{saison}-spieltag/{journee}/"
            response = requests.get(url)
            if response.status_code != 200:
                break

            soup = BeautifulSoup(response.text, "html.parser")
            rows = soup.select("table tr")
            if not rows:
                break

            last_date = None
            for row in rows:
                cols = row.find_all("td")
                if len(cols) < 6:
                    continue

                try:
                    date = cols[0].get_text(strip=True)
                    if date:
                        last_date = date
                    else:
                        date = last_date

                    # Conversion en format ISO (YYYY-MM-DD)
                    try:
                        date_iso = datetime.strptime(date, "%d.%m.%Y").strftime("%Y-%m-%d")
                    except Exception:
                        date_iso = ""

                    team1 = cols[2].get_text(strip=True)
                    team2 = cols[4].get_text(strip=True)
                    score_raw = cols[5].get_text(strip=True)

                    if ":" in score_raw and "-" not in score_raw:
                        score_clean = score_raw.split()[0]
                        score1, score2 = score_clean.split(":")
                        all_matches.append([date_iso, ligue_url, journee, team1, team2, score1, score2])
                except Exception:
                    continue

            print(f"✅ Journée {journee} de {ligue} {saison} scrapée")
            journee += 1

    # --- Trier par date ---
    def parse_date(d):
        try:
            return datetime.strptime(d, "%Y-%m-%d")
        except Exception:
            return datetime.max

    all_matches.sort(key=lambda x: parse_date(x[0]))

    # --- Sauvegarde CSV ---
    with open(name_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        #writer.writerow(["date", "ligue", "journee", "team1", "team2", "score1", "score2"])
        writer.writerows(all_matches)

    print(f"\n🎉 Fichier unique créé et trié : {name_file}")


In [2]:
scrap_saison_complete("2025-2026",10)

# max journée correspond à l'avancement maximum des championnats


✅ Journée 1 de ligue1 2025-2026 scrapée
✅ Journée 2 de ligue1 2025-2026 scrapée
✅ Journée 3 de ligue1 2025-2026 scrapée
✅ Journée 4 de ligue1 2025-2026 scrapée
✅ Journée 5 de ligue1 2025-2026 scrapée
✅ Journée 6 de ligue1 2025-2026 scrapée
✅ Journée 7 de ligue1 2025-2026 scrapée
✅ Journée 8 de ligue1 2025-2026 scrapée
✅ Journée 9 de ligue1 2025-2026 scrapée
✅ Journée 10 de ligue1 2025-2026 scrapée
✅ Journée 1 de ligue2 2025-2026 scrapée
✅ Journée 2 de ligue2 2025-2026 scrapée
✅ Journée 3 de ligue2 2025-2026 scrapée
✅ Journée 4 de ligue2 2025-2026 scrapée
✅ Journée 5 de ligue2 2025-2026 scrapée
✅ Journée 6 de ligue2 2025-2026 scrapée
✅ Journée 7 de ligue2 2025-2026 scrapée
✅ Journée 8 de ligue2 2025-2026 scrapée
✅ Journée 9 de ligue2 2025-2026 scrapée
✅ Journée 10 de ligue2 2025-2026 scrapée
✅ Journée 1 de national 2025-2026 scrapée
✅ Journée 2 de national 2025-2026 scrapée
✅ Journée 3 de national 2025-2026 scrapée
✅ Journée 4 de national 2025-2026 scrapée
✅ Journée 5 de national 2025-2